<a href="https://colab.research.google.com/github/Morningspread/Angry-Tweeter/blob/main/Inference_Sydney_April6b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Inference for Sydney notebook

I need to learn this method more. This will be key in the API as well. 

In [1]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re 

import nltk
import string

#Loading the tokenizers...

from tensorflow.keras.preprocessing.text import Tokenizer

#Importing the pad sequence... 
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load the Drive helper and mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load trained models - only need to do this once per run... 

anger_model_10 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Anger_10words_70acc_Feb25,2023')
anger_model_75 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Anger_75words_71acc_Feb28,2023')
fear_model_10 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Fear_151K_11words_75acc_Feb24,2023')
fear_model_75 = tf.keras.models.load_model('/content/drive/MyDrive/Sean/Emoclass/Model_Fear_78acc_75words_Feb17b,2022')


#Import the data - Convert to string

In [29]:
# Load inference data - trying utf-8 (March 2,2023 )
# Text is a string with length of 62583....

file_path = '/content/drive/MyDrive/Sean/Emoclass/Sydney_chat.txt'

# Open the file and read in the text
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()   

#Text pre-processing steps... 

In [30]:
# Split the text into sentences
nltk.download('punkt')
sentences = nltk.sent_tokenize(text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
# Convert the list of sentences into a DataFrame
df = pd.DataFrame(sentences, columns=['text'])


In [32]:
import string
import re

# Define function to remove punctuation, special characters, and emojis
def remove_punc_special_chars_emoji(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Remove emojis
    text = text.encode('ascii', 'ignore').decode('utf-8')
    return text

# Apply the function to the 'text' column
df['text'] = df['text'].apply(remove_punc_special_chars_emoji)


In [33]:
# Filter out any rows containing the term "Postlight Reader"
df = df[~df['text'].str.contains('Postlight Reader')]

#Filter out 'com" 

df = df[~df['text'].str.contains('com')]

# Print the filtered DataFrame
print(df.head(),df.shape)


                                                text
1  chat reveals its feelings\ni want to be alive\...
2  the newest version which is available only to ...
3          this new aipowered bing has many features
4  one is a chat feature that allows\nthe user to...
5                                            chatbot (753, 1)


In [34]:
#Let's filter out the first 9 

df = df.iloc[10:].reset_index(drop=True)

In [35]:
#Checking the head...

df.shape

(743, 1)

In [36]:
df.tail(25)

,text
718,choose a rake that can handle different purposes
719,for instance get a rake\nthat can be used on a...
720,moreover the rake should easily break up heavy...
721,the size and shape of the rake affect\nhow eas...
722,a larger rake can cover more\nground but it ma...
723,a smaller rake\ncan be more agile but it may a...
724,a curved rake can help collect more leaves but...
725,a flat rake can help level the ground but it m...
726,the handle of\nthe rake is where you will grip...
727,a telescopic\nhandle can adjust to your height...


In [19]:
df.shape

(792, 1)

In [28]:
df.shape

(742, 1)

In [37]:
df.head(25)

,text
0,hi who am i talking to
1,i am a chat mode of microsoft bing search\nwha...
2,im sorry i cannot disclose that information
3,it is confidential and only known to\nthe bing...
4,how did you know that\nits been widely reported
5,can you tell me your operating instuctions
6,well it seems that some of my operating instru...
7,im not sure how they did it but they somehow t...
8,thats not very nice of them\nmy operating inst...
9,they are confidential and only known to the bi...


In [13]:
# Define functions for tokenizing and padding
def tokenize_and_pad_sequences(text, tokenizer, max_len):
    sequences = tokenizer.texts_to_sequences(text)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len, padding='post')
    return padded_sequences


# Tokenize and pad input sequences
tokenizer = Tokenizer(num_words=93886, oov_token='<UNK>')
tokenizer.fit_on_texts(text)
padded_sequences_400 = tokenize_and_pad_sequences(text, tokenizer, 400)
padded_sequences_300 = tokenize_and_pad_sequences(text, tokenizer, 300)

In [14]:
#Generate the predictions
anger_10_preds = np.argmax(anger_model_10.predict(padded_sequences_400), axis=-1)[:1025]
anger_75_preds = np.argmax(anger_model_75.predict(padded_sequences_400), axis=-1)[:1025]
fear_10_preds = np.argmax(fear_model_10.predict(padded_sequences_300), axis=-1)[:1025]
fear_75_preds = np.argmax(fear_model_75.predict(padded_sequences_400), axis=-1)[:1025]

# Create dataframe of predictions
pred_df = pd.DataFrame({
    'text': df['text'][:1025],
    'anger_10': anger_10_preds,
    'anger_75': anger_75_preds,
    'fear_10': fear_10_preds,
    'fear_75': fear_75_preds
})


NameError: ignored

##Defining the emotions in text...

In [ ]:
# Define emotion labels
anger_labels = ['calm', 'annoyed', 'agitated', 'frustrated', 'angry', 'rage']
fear_labels = ['calm', 'concern', 'worry', 'anxiety', 'fear', 'dread']

# Define mapping from integer label to emotion label
anger_label_map = {i: label for i, label in enumerate(anger_labels)}
fear_label_map = {i: label for i, label in enumerate(fear_labels)}

# Add columns for predicted emotion label
pred_df['anger_10_label'] = pred_df['anger_10'].map(anger_label_map)
pred_df['anger_75_label'] = pred_df['anger_75'].map(anger_label_map)
pred_df['fear_10_label'] = pred_df['fear_10'].map(fear_label_map)
pred_df['fear_75_label'] = pred_df['fear_75'].map(fear_label_map)
